In [1]:
# Liturgický kalendár:
# 365 dní ako "one page" zobrazenie.

In [2]:
import os
import copy
from datetime import datetime
from bs4 import BeautifulSoup, Comment

In [3]:
# kde sú html súbory:
data_dir = './data_html/'

In [4]:
# List all html files from data directory with file name length 15 (meaning particular date like: '2023-01-01.html')
# poznámka: 
# keďže cirkevný rok začína adventom, sú v zip archíve aj dni z predošlého roka (2022)
# preto som pre názornosť dal podmienku, aby bral len aktuálny rok 2023 (file.startswith('2023'))
html_files = [ file for file in os.listdir(data_dir) if file.startswith('2023') and file.endswith(".html") and len(file)==15 ]

len(html_files) 

365

In [5]:
fname = html_files[0]

with open(data_dir+fname, encoding='utf-8') as f_in:
    s = f_in.read()      

soup1 = BeautifulSoup(s,'lxml') # 
soup1.title.string = 'kalendár 2023' # meníme title
init_script = copy.copy(soup1.body.script)
soup1.body.clear()
soup1.body.append(init_script)

my_table = soup1.new_tag('table')
my_table['border']="1" # 
# my_table = BeautifulSoup('<table border="1"></table>', 'html.parser') # tu musí byť 'html.parser' !
soup1.body.append(my_table)

c:\Python\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [6]:
for fname in html_files:

    with open(data_dir+fname, encoding='utf-8') as f_in:
        s = f_in.read()            

    date_string = fname[:-5]
    date_object = datetime.strptime(date_string, '%Y-%m-%d')
    tyzden = date_object.strftime('%V')
    
    if tyzden.startswith('0'):
        tyzden = tyzden[1:]

    new_table_row = soup1.new_tag('tr')
    # new_table_row['id'] = date_string # id='2023-01-01' napríklad
    soup1.table.append(new_table_row)
        
    if date_string=='2023-01-01':
        td_tyzden = soup1.new_tag('td')
        rowspan = ((date_object.weekday()+1) % 7)+1
        td_tyzden['rowspan'] = str(rowspan)
        td_tyzden.string = f'{tyzden}.\u00a0týždeň'
        # tyzden +=1
        new_table_row.append(td_tyzden)        

    elif (date_object.weekday() == 0): # pondelok
        td_tyzden = soup1.new_tag('td')
        td_tyzden['rowspan'] = "7"
        td_tyzden.string = f'{tyzden}.\u00a0týždeň'
        # tyzden +=1
        new_table_row.append(td_tyzden)

    ### soup2
    ### 
    soup2 = BeautifulSoup(s,'lxml') #    
    body = soup2.body
    
    body.script.extract() # init(); preč

    elems = body.find_all('div', class_='lcHEADinfo') # mažeme súradnice
    for elem in elems: #  v príslušný deň ich môže byť viac
        elem.extract()        
    
    elems = body.find_all('div', class_='lcDENalt') # vigília, mažeme
    if 'psv' in elems[0]['class']: # v prvom nájdenom rušíme štýlovanie (druhý aj tak zmažeme)
        elems[0]['class'].remove('psv') # prikázaný sviatok štýlovanie rušíme (červený okraj)
    if len(elems) == 2:        # vigília je vtedy, ak máme dvakrát lcDENalt       
        elems[1].extract()     # vigíliu mažeme     

    elems = body.find_all('div', class_='lcBODY') # čítania mažeme
    for elem in elems: #  v príslušný deň ich môže byť viac
        elem.extract()        

    # body.find('div', class_='lcNAVIG').extract()    
    body.div.section.div.extract() # <div class="lcNAVIG"> # navigačné tlačidlá mažeme    

    povodny_link = body.h1.a['href']
    body.h1.a['href'] = 'onepage_tabulka.html#'+povodny_link[:-5] # upravujeme link

    body.h1.span.find('span', class_='lcWD').append(soup2.new_tag('br'))

    what = body.h1.find('a', class_='lcMY')
    what.string = what.string[:-5] # mažeme rok 2023 z textu

    body.h1.span.find('span', class_='lcDMY').append(soup2.new_tag('br'))
    
    for elem in body.find_all('h1'): # nechceme nadpisy, meníme na div
        elem.name = 'div'

    for elem in body.find_all('h2'): # nechceme nadpisy, meníme na div
        elem.name = 'div'

    td1 = soup1.new_tag('td')
    td1['id'] = date_string # id='2023-01-01' napríklad
    new_table_row.append(td1) # <tr><td>
    what = body.find('span', class_='lcDMY')
    td1.append(what) # appending is also moving...   
    what = body.find('span', class_='lcWD')
    td1.append(what) # appending is also moving...

    td2 = soup1.new_tag('td')
    new_table_row.append(td2) # <tr><td>
    what = body.find('span', class_='lcND')
    td2.append(what) # appending is also moving...
    td2.append(body.div) # appending the rest (which is <div class="lc hvr-scl-chldrn-a">)

In [7]:
# removing html comments <!-- -->
comments = soup1.find_all(string = lambda x: isinstance(x, Comment))
for comment in comments:
    comment.extract()

# removing tags with no content https://stackoverflow.com/questions/33500888/how-to-remove-tags-that-have-no-content
for x in soup1.find_all():
    if (len(x.get_text(strip=True)) == 0) and (x.name not in ['br', 'img', 'meta', 'link', 'script']):
        x.extract()

In [8]:
with open('onepage.html', mode='w', encoding='utf-8') as f_out:
    f_out.write(str(soup1))

In [ ]:
# done